# Bootstrapping user preferences



Similarly to (Zhu et al., 2021), we'll evaluate a dynamic recommendation system using:

1. An oracle preference model (Generated in notebook 00-preference_model)
2. A position bias examination model
3. Bootstrapped user preferences

Step 3 is important so that we're simulating exposing users to fresh items, using their previous preferences (provided by step 1) and their examination bias (step 2) to determine a sinthethical rating matrix dataset that we'll use to compare different calibration approaches as the user gets exposed to more and more items.

In [30]:
import pandas as pd
from tqdm import tqdm

In [31]:
import numpy as np
import math
import random

In [32]:
import sys
import os
from pathlib import Path

sys.path.append('..')

In [33]:
from constants import ML_1M_FILLED_PATH

In [34]:
def get_user_preference_for_item(user, item, matrix):
    user_ratings = matrix[matrix["user"] == user]
    return user_ratings[user_ratings["item"] == item].rating.item()

In [35]:
def click_model(k):
    lambda_k = 1/math.log(k+1,2)
    examination_probability = random.random()
    if examination_probability <= lambda_k:
        return True
    return False

In [36]:
def get_inverse_propensity_click_score(position):
    # Given a click position, this funtion returns the invense propensity, 
    # usefull to debias the data later.
    return - 1/math.log(position+1,2)

In [ ]:
def get_user_feedback_for_item(user, item ,k, matrix):
    preference = get_user_preference_for_item(user, item, matrix)
    observed = click_model(k)
    relevant = bool(preference)
    should_click = observed and relevant
    if (should_click):
        feedback = 1
        clicked_at = k
    else:
        if (observed):
            # Case where an item was observed, but isn´t relevant -> negative example for BPR
            feedback = 0
        else:
            # Case where an item was neither observed or relevant -> we will ignore this training instance in this loop
            feedback = None
        clicked_at = None
        
    # If user clicked the item, record the position it was in
    # feedback = 1 if user examined and clicked, 0 if user examined and not clicked,
    # None if otherwise
    return user, item, feedback, clicked_at

In [38]:
def map_recommendation_to_feedback(user, rec_list, matrix):
    return [get_user_feedback_for_item(user, item, idx+1, matrix) for idx, item in enumerate(rec_list)]

In [39]:
def random_rec(items, u, k, D):
    user_history = set(D[D["user"] == u]["item"])
    candidate_items = list(set(items) - user_history)
    return random.sample(candidate_items, k)

In [40]:
preference_matrix = pd.read_csv(f"../{ML_1M_FILLED_PATH}").drop(columns=["Unnamed: 0"])

In [41]:
unique_users = list(preference_matrix["user"].unique())

unique_items = list(preference_matrix["item"].unique())

In [42]:
click_matrix = pd.DataFrame(columns=["user", "item", "relevancy", "clicked_at"])

In [43]:
def bootstrap_clicks(D, unique_users, unique_items, preference_matrix, k=20):
    """
    Given unique users and unique items, recommend up to k items to every user
    using a preference matrix as a relevancy model and using a click model
    to simulate probability of user examinating an item.

    Feedback signal will be fed to the D matrix.
    """
    rows_to_append = []
    for user in tqdm(unique_users, desc="Processing users..."):
        recs = random_rec(unique_items, user, k, D)
        row = map_recommendation_to_feedback(user, recs,preference_matrix)
        rows_to_append.extend(row) 

    new_df = pd.DataFrame(rows_to_append, columns=["user", "item", "relevancy", "clicked_at"])
    return pd.concat([D, new_df])

In [44]:
%%time
click_matrix = bootstrap_clicks(click_matrix, unique_users, unique_items, preference_matrix)

Processing users...: 100%|██████████| 6040/6040 [18:17<00:00,  5.50it/s]


CPU times: user 18min 17s, sys: 764 ms, total: 18min 18s
Wall time: 18min 17s


/tmp/ipykernel_7989/2389138707.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([D, new_df])


In [45]:
click_matrix

,user,item,relevancy,clicked_at
0,1,3183,1.0,1.0
1,1,1375,NaN,NaN
2,1,2162,NaN,NaN
3,1,353,NaN,NaN
4,1,2519,NaN,NaN
...,...,...,...,...
120795,6040,3248,NaN,NaN
120796,6040,3937,0.0,NaN
120797,6040,3423,NaN,NaN
120798,6040,3246,NaN,NaN


In [46]:
click_matrix.groupby(["relevancy", "clicked_at"], dropna=False).sum()

user       item
relevancy clicked_at                      
0.0       NaN          86585013   57450238
1.0       1.0           6000537    3879410
          2.0           3703735    2457692
          3.0           2871579    1922135
          4.0           2544748    1698324
          5.0           2254124    1462630
          6.0           2197165    1488362
          7.0           1973326    1255953
          8.0           1924512    1221810
          9.0           1683051    1112390
          10.0          1619060    1058129
          11.0          1767116    1058868
          12.0          1544126    1051881
          13.0          1538499    1004681
          14.0          1530206    1062276
          15.0          1396617     987987
          16.0          1219887     858098
          17.0          1460037     967548
          18.0          1430164     924181
          19.0          1258535     844838
          20.0          1427702     865701
NaN       NaN         236946661  156504161

In [48]:
click_matrix.to_csv("../data/simulation/sinthetic_data.csv", index=False)

In [49]:
click_matrix = pd.read_csv("../data/simulation/sinthetic_data.csv")

In [50]:
click_matrix

,user,item,relevancy,clicked_at
0,1,3183,1.0,1.0
1,1,1375,NaN,NaN
2,1,2162,NaN,NaN
3,1,353,NaN,NaN
4,1,2519,NaN,NaN
...,...,...,...,...
120795,6040,3248,NaN,NaN
120796,6040,3937,0.0,NaN
120797,6040,3423,NaN,NaN
120798,6040,3246,NaN,NaN
